# Live Squat Checker using Frame Analysis

Live realtime analysis by heuristic technique.

Squat instructions: Stand side on with right-side facing towards the camera. Get into the starting position, squeezing your shoulder blades back and squat down.

Avoid:
- Leaning forwards at any point
- Half-squatting, make sure you go parallel to get the full range of motion

<div>

</div>
https://en.wikipedia.org/wiki/Squat_(exercise)

## TOC:
0. [Setup](#0.-Import-Libraries-&-Setup)
1. [Draw Angles](#1.-Draw-Angles-&-Display)

# 0. Import Libraries & Setup

In [6]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import math
from joblib import dump, load

In [7]:
# Load model and choose model to use
mlp_model = load('mlp-squat-mistakes.joblib')
mnb_model = load('mnb-squat-mistakes.joblib')
svc_model = load('svc-squat-mistakes.joblib')

model = mnb_model

In [8]:
def get_landmark(landmarks):
    unboxed_landmark = {}
    
    for i in range(33):
        unboxed_landmark['x' + str(i)] = landmarks.landmark[i].x
        unboxed_landmark['y' + str(i)] = landmarks.landmark[i].y
        unboxed_landmark['z' + str(i)] = landmarks.landmark[i].z
    
    df = pd.DataFrame([unboxed_landmark])    
    
    return df

# 1. Display

In [9]:
# Get the angle between 3 keypoint's coordinates
# INPUT: frame (dictonary storing the coordinates of all the keypoints), the keypoint numbers
# OUTPUT: angle as float
def get_angle(frame, keypoint_1, keypoint_2, keypoint_3):
    a = np.array([frame['x'+str(keypoint_1)], frame['y'+str(keypoint_1)]]) # First
    b = np.array([frame['x'+str(keypoint_2)], frame['y'+str(keypoint_2)]]) # Mid
    c = np.array([frame['x'+str(keypoint_3)], frame['y'+str(keypoint_3)]]) # End
    
    ba = a - b
    bc = c - b

    ba = [j for i in ba for j in i]
    bc = [j for i in bc for j in i]
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = math.degrees(np.arccos(cosine_angle))
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle


# Get the y differnece between two keypoints in a frame
# INPUT: frame (dictionary of keypoints to coordinates) and the 2 keypoints
# OUPUT: absolute value of the difference
def get_y_difference(frame, keypoint_1, keypoint_2):
    return abs(frame['y'+str(keypoint_1)] - frame['y'+str(keypoint_2)])

In [10]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, image = cap.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        display_msg = 'Cannot detect skeleton in frame'
        
        if results.pose_landmarks != None:
            # Setup heuristic values for input into model
            heuristics = {}

            heuristics['hip_angle'] = get_angle(get_landmark(results.pose_landmarks), 11, 23, 25)
            heuristics['knee_hip_difference'] = get_y_difference(get_landmark(results.pose_landmarks), 23, 25)
            heuristics['knee_angle'] = get_angle(get_landmark(results.pose_landmarks), 23, 25, 27)
            
            # Get the ML model's prediction
            prediction = (model.predict(pd.DataFrame([heuristics]))[0])
            
            # Set the display message
            if prediction == 'parallel':
                display_msg = 'Make sure you go parallel!'
            elif prediction == 'leaning':
                display_msg = 'Keep your back straight'
            elif prediction == 'good':
                display_msg = 'Good so far, keep it up'
        
        # Show the display message on the UI
        cv2.rectangle(image, (0, 0), (900, 60), (0, 0, 0), -1)
        cv2.putText(image, display_msg, (0,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        cv2.namedWindow('output', cv2.WINDOW_NORMAL) # Allows you to manually change the size of the window
        cv2.imshow('output', image)

        # Press ESC to exit
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()